# 🧠 Project: Vision Transformer (ViT) Classifier for Medical or Satellite Imagery
This notebook walks through fine-tuning a pre-trained Vision Transformer (ViT) on a medical or satellite image classification task using Hugging Face Transformers and PyTorch.

In [ ]:
!pip install transformers datasets torchvision timm

In [ ]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from datasets import load_dataset
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import os

In [ ]:
# Load EuroSAT (satellite imagery) dataset
dataset = load_dataset('eurosat', split='train[:1000]')
dataset = dataset.train_test_split(test_size=0.2)
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

In [ ]:
def transform(example):
    image = example['image'].convert('RGB')
    example['pixel_values'] = feature_extractor(images=image, return_tensors='pt')['pixel_values'][0]
    return example

dataset = dataset.map(transform)

In [ ]:
dataset.set_format(type='torch', columns=['pixel_values', 'label'])

In [ ]:
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=dataset['train'].features['label'].num_classes
)

In [ ]:
training_args = TrainingArguments(
    output_dir='./vit-results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    save_steps=10,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

## ✅ Summary
This notebook demonstrates how to fine-tune a Vision Transformer on satellite imagery using the EuroSAT dataset. You can swap the dataset for medical images or fine-tune on your own labeled data.